# Credibility Sub-Agent Test Notebook

This notebook tests the credibility sub-agent, which is responsible for:
- **Claim Verification**: Checking if claims are supported by reliable evidence
- **Source Assessment**: Evaluating the trustworthiness of sources used
- **Consistency Checking**: Looking for contradictions or inconsistencies
- **Bias Identification**: Noting potential biases in sources or analysis

The main agent uses this sub-agent to:
- Verify research findings from the web research agent
- Assess source quality and reliability
- Identify potential misinformation or biased reporting
- Rate overall trustworthiness of gathered information

## Setup

In [1]:
# Ensure scratchpad folders exist and are empty
from pathlib import Path
import shutil

scratchpad = Path("../scratchpad")
for folder in ["data", "images", "notes", "plots", "reports"]:
    path = scratchpad / folder
    if path.exists():
        shutil.rmtree(path)
    path.mkdir(parents=True)
    
print("✨ Scratchpad folders ready (data, images, notes, plots, reports)")


✨ Scratchpad folders ready (data, images, notes, plots, reports)


In [2]:
import sys
sys.path.insert(0, '..')

from agents.credibility_agent import credibility_agent_graph
from dotenv import load_dotenv

load_dotenv()

print("✅ Credibility agent loaded")


✓ Scratchpad directories cleared: data, images, notes, plots, final
✅ Credibility agent loaded


## Helper Function to Test the Agent

In [3]:
from langchain_core.messages import AIMessage, ToolMessage
from IPython.display import display, Markdown

def truncate(text, limit=2000):
    return text[:limit] + "\n..." if len(text) > limit else text

def test_credibility_agent(message: str, thread_id: str = None):
    """Run the credibility agent and display all intermediate steps."""
    thread_id = thread_id or f"test-{__import__('time').time()}"

    display(Markdown(f"## 📝 Task\n```\n{message.strip()}\n```\n---"))

    step = 0
    for event in credibility_agent_graph.stream(
        {"messages": [("user", message)]},
        config={"configurable": {"thread_id": thread_id}},
        stream_mode="updates"
    ):
        for node_name, node_output in event.items():
            messages = node_output.get("messages", [])
            for msg in messages:
                if isinstance(msg, AIMessage) and msg.tool_calls:
                    step += 1
                    for tc in msg.tool_calls:
                        name, args = tc.get("name"), tc.get("args", {})
                        if name == "execute_python" and "code" in args:
                            display(Markdown(f"### 🔧 Tool Call: `{name}`\n```python\n{truncate(args['code'], 1500)}\n```"))
                        else:
                            display(Markdown(f"### 🔧 Tool Call: `{name}`\n```json\n{truncate(str(args), 500)}\n```"))

                elif isinstance(msg, ToolMessage):
                    display(Markdown(f"### 📤 Tool Response\n```\n{truncate(msg.content)}\n```\n---"))

                elif isinstance(msg, AIMessage) and msg.content and not msg.tool_calls:
                    display(Markdown(f"## ✅ Response\n{msg.content}"))
                    return msg.content


---
# Example 1: Simple Claim Verification (Simple)

**Context**: A simple factual claim about a company that needs verification.

**Sub-agent role**: Verify a straightforward claim using web search to find corroborating evidence.

In [4]:
# Example 1: Simple claim verification
example_1_message = """Please verify the following claim:

"Apple Inc. is the most valuable publicly traded company in the world by market capitalization."

Check if this claim is accurate as of the current date.
"""

# Use a fresh thread ID to avoid any cached responses
import time
response_1 = test_credibility_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

## 📝 Task
```
Please verify the following claim:

"Apple Inc. is the most valuable publicly traded company in the world by market capitalization."

Check if this claim is accurate as of the current date.
```
---

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/website_quality.txt', 'limit': 200}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/research_lessons.txt', 'limit': 200}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/source_notes.txt', 'limit': 200}
```

### 🔧 Tool Call: `web_search`
```json
{'query': 'largest publicly traded company by market capitalization December 2025', 'max_results': 3, 'topic': 'finance'}
```

...

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/research_lessons.txt', 'limit': 200}
```

### 📤 Tool Response
```
     1	## Research & Analysis Lessons
     2	- Use clear, minimal matplotlib styles and always report saved figure paths for users.
```
---

### 🔧 Tool Call: `edit_file`
```json
{'file_path': '/memories/research_lessons.txt', 'old_string': '## Research & Analysis Lessons\n- Use clear, minimal matplotlib styles and always report saved figure paths for users.', 'new_string': '## Research & Analysis Lessons\n- Use clear, minimal matplotlib styles and always report saved figure paths for users.\n- For “largest company by market cap” checks, prefer same-day rankings from major financial data providers (Bloomberg, Refinitiv, Yahoo Finance) and note that leadership can change 
...
```

### 📤 Tool Response
```
Successfully replaced 1 instance(s) of the string in '/research_lessons.txt'
```
---

## ✅ Response
### Claim Verification
- **Claim**: “Apple Inc. is the most valuable publicly traded company in the world by market capitalization.”
  - **Status**: **UNVERIFIED / LIKELY OUTDATED**
  - **Evidence**:  
    - Recent rankings (e.g., Investopedia’s “Biggest Companies in the World by Market Cap”) list **Apple, Nvidia, and Microsoft** as the three largest companies by value, all above $3T, but do **not** clearly state that Apple is consistently number one at present; rather, these three trade places as markets move.  
    - Market cap leadership among Apple, Microsoft, Nvidia (and sometimes Saudi Aramco) has shifted frequently in 2024–2025, often changing **intra‑day**. Current finance pages (Yahoo Finance for AAPL, MSFT, NVDA) show all around multi‑trillion valuations, so without a precise timestamp and real‑time feed, it is not possible to affirm that Apple is definitively the largest **right now**.  
  - **Conclusion**: Apple is *one of* the most valuable companies and has held the top spot at various times, but the statement “is the most valuable” at this exact date/time cannot be reliably confirmed from static sources; it should be treated as **time‑sensitive and likely inaccurate as a blanket statement**.

### Source Assessment
- **Overall source quality**: **4/5**
  - Finance data from **Investopedia** and **Yahoo Finance** are generally high‑credibility for market cap figures.
- **Concerns**:
  - Rankings move quickly; articles may be **weeks or months old**.
  - No single static article can guarantee who is #1 “as of now” without a real‑time data feed.

### Answer Quality
- **Does it answer the original question?** **Partially**
  - I can say the claim is **not safely accurate as a present‑tense generalization**.
  - I cannot provide a precise real‑time ranking at this second.
- **Missing elements**:
  - Exact, timestamped market cap numbers for Apple vs Microsoft/Nvidia/Saudi Aramco pulled from a live quote system.

### Recommendations
- Rephrase the claim to something like:
  - “Apple is one of the most valuable publicly traded companies in the world by market capitalization and has at times been the largest.”
- For strict verification:
  - Check a **real‑time finance terminal or live quote service** (Bloomberg, Refinitiv, major broker platform) and compare Apple’s market cap against Microsoft, Nvidia, and Saudi Aramco at the exact time of interest.

### Final Verdict
- **Trustworthy and defensible?** **With caveats**
  - The claim is **too absolute** and **time‑sensitive** to be considered reliably true “as of now” without live data; treat it as **potentially outdated**.

---
# Example 2: Research Output Assessment (Medium)

**Context**: The web research agent has produced findings about a stock that need verification.

**Sub-agent role**: Assess the credibility of multiple claims and sources from a research report.

In [5]:
# Example 2: Research output assessment
example_2_message = """Please assess the credibility of this research output:

---
## NVIDIA Stock Analysis Summary

**Key Findings:**
1. NVIDIA's data center revenue grew over 200% year-over-year in their most recent quarter
2. The company has captured approximately 80-90% of the AI chip market
3. Major tech companies including Microsoft, Google, and Amazon are all using NVIDIA GPUs for AI workloads
4. The stock has risen over 200% in 2024

**Sources Used:**
- NVIDIA Investor Relations
- Reuters
- Bloomberg
- TechCrunch

**Original Question:** "Why has NVIDIA stock performed so well recently?"
---

Verify the key claims and assess whether this research adequately answers the original question.
"""

response_2 = test_credibility_agent(example_2_message, thread_id=f"example-2-{int(time.time())}")

## 📝 Task
```
Please assess the credibility of this research output:

---
## NVIDIA Stock Analysis Summary

**Key Findings:**
1. NVIDIA's data center revenue grew over 200% year-over-year in their most recent quarter
2. The company has captured approximately 80-90% of the AI chip market
3. Major tech companies including Microsoft, Google, and Amazon are all using NVIDIA GPUs for AI workloads
4. The stock has risen over 200% in 2024

**Sources Used:**
- NVIDIA Investor Relations
- Reuters
- Bloomberg
- TechCrunch

**Original Question:** "Why has NVIDIA stock performed so well recently?"
---

Verify the key claims and assess whether this research adequately answers the original question.
```
---

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/website_quality.txt', 'limit': 200}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/research_lessons.txt', 'limit': 200}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/source_notes.txt', 'limit': 200}
```

### 📤 Tool Response
```
     1	## Website Quality Notes
     2	- Finance-focused sites like Yahoo Finance and Investopedia are generally reliable for up-to-date market cap rankings but should be cross-checked for dates and definitions.
```
---

...

### 🔧 Tool Call: `edit_file`
```json
{'file_path': '/memories/research_lessons.txt', 'old_string': '## Research & Analysis Lessons\n- Use clear, minimal matplotlib styles and always report saved figure paths for users.\n- For “largest company by market cap” checks, prefer same-day rankings from major financial data providers (Bloomberg, Refinitiv, Yahoo Finance) and note that leadership can change intraday.', 'new_string': '## Research & Analysis Lessons\n- Use clear, minimal matplotlib styles and always report saved figure paths f
...
```

### 📤 Tool Response
```
Successfully replaced 1 instance(s) of the string in '/research_lessons.txt'
```
---

### 🔧 Tool Call: `edit_file`
```json
{'file_path': '/memories/source_notes.txt', 'old_string': '## Source Notes\n- When checking "largest company by market cap" claims, confirm the exact date and whether intraday or closing market cap is used.', 'new_string': '## Source Notes\n- When checking "largest company by market cap" claims, confirm the exact date and whether intraday or closing market cap is used.\n- Nvidia-related claims are often time-sensitive; always confirm the specific quarter or year referenced in financial stats bef
...
```

### 📤 Tool Response
```
Successfully replaced 1 instance(s) of the string in '/source_notes.txt'
```
---

## ✅ Response
### Claim Verification
- **Claim 1: “NVIDIA's data center revenue grew over 200% year-over-year in their most recent quarter”**  
  - **Status:** PARTIALLY VERIFIED  
  - **Evidence:** Recent earnings (e.g., FY2024 and 2024 calendar quarters) show data center revenue growing well above 100% YoY, but not consistently “over 200%” in the very latest reported quarter; some quarters were ~170–190% YoY. The “most recent quarter” wording is likely overstated unless anchored to a specific older quarter.

- **Claim 2: “The company has captured approximately 80-90% of the AI chip market”**  
  - **Status:** PARTIALLY VERIFIED  
  - **Evidence:** Multiple reports (Reuters, others) describe NVIDIA having a dominant share of the AI accelerator/GPU market, often cited around 70–90%. Exact percentages vary by definition (GPU vs. all AI accelerators, unit vs. revenue share). Directionally correct, but 80–90% should be treated as an estimate, not a hard fact.

- **Claim 3: “Microsoft, Google, and Amazon are all using NVIDIA GPUs for AI workloads”**  
  - **Status:** VERIFIED  
  - **Evidence:** Public cloud offerings (Azure, Google Cloud, AWS) all list NVIDIA GPUs (e.g., A100, H100, L4, etc.) for AI/ML instances, and this is widely documented in their own product pages and tech press.

- **Claim 4: “The stock has risen over 200% in 2024”**  
  - **Status:** UNVERIFIED / LIKELY INACCURATE AS STATED  
  - **Evidence:** Depending on the exact start and end dates used, NVIDIA has had very large gains, but a >200% increase *within the single calendar year 2024* is not supported by typical price data; much of the multi-hundred-percent gain is over a multi‑year period. The claim appears to conflate multi‑year returns with 2024 YTD.

### Source Assessment
- **Overall source quality:** 4/5  
- **Concerns:**  
  - The listed outlets (NVIDIA IR, Reuters, Bloomberg, TechCrunch) are generally high‑credibility.  
  - However, the summary does not show specific article dates, quotes, or figures, which makes the timeframes ambiguous and inflates some numbers.

### Answer Quality
- **Does it answer the original question (“Why has NVIDIA stock performed so well recently?”)?**  
  - **Partially.**  
- **Missing elements:**  
  - Links between facts and the stock move are only implicit. It should explicitly connect:  
    - Explosive data center/AI revenue growth → earnings surprises → multiple expansion.  
    - Near‑monopoly in AI accelerators → strong pricing power and visibility into future demand.  
    - Broad hyperscaler adoption (MSFT/GOOGL/AMZN) → durability of demand.  
  - No discussion of valuation risks, competition (e.g., AMD, custom chips), or macro factors (rates, tech rally) that also affect performance.

### Recommendations
- Quantify with precise, dated figures (e.g., “Data center revenue grew X% YoY in Qx FY2024 per NVIDIA IR”) rather than rounded “200%+”.  
- Correct or qualify the “>200% in 2024” stock performance claim; specify timeframe (e.g., “over the last two years”).  
- Add one or two sentences explicitly tying each driver (revenue growth, market share, cloud adoption) to investor expectations and share price behavior.

### Final Verdict
- **Trustworthy and defensible?** With caveats  
  - Directionally accurate on drivers and broad market position, but some key numbers and timeframes are overstated or imprecise.

---
# Example 3: Contradictory Information Assessment (Complex)

**Context**: Research has uncovered conflicting information about a stock from different sources.

**Sub-agent role**: Analyze contradictory claims, assess source reliability, and determine which narrative is more credible.

In [6]:
# Example 3: Contradictory information assessment
example_3_message = """Please assess the credibility of these conflicting research findings:

---
## Tesla Stock Analysis - Conflicting Views

**Bullish Research (Source: Tesla fan blog "TeslaDaily.com"):**
- "Tesla will dominate the robotaxi market by 2025"
- "Full Self-Driving is already safer than human drivers"
- "Tesla's margins are industry-leading and sustainable"
- "Competition from legacy automakers is irrelevant"

**Bearish Research (Source: Short-seller report from "Hindenburg Research"):**
- "Tesla's FSD claims are exaggerated and potentially dangerous"
- "Margins are declining due to price cuts and competition"
- "Chinese EV makers are taking significant market share"
- "Robotaxi timeline has been repeatedly delayed"

**Neutral Research (Source: Goldman Sachs equity research):**
- "Tesla maintains technology leadership but faces margin pressure"
- "FSD progress is notable but regulatory approval timeline uncertain"
- "Competition is intensifying but Tesla's brand remains strong"

**Original Question:** "What is the outlook for Tesla stock over the next 12 months?"
---

Please:
1. Assess the credibility of each source
2. Identify which claims can be verified vs which are speculative
3. Note any potential biases in each source
4. Determine if this research provides a balanced view to answer the original question
"""

response_3 = test_credibility_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

## 📝 Task
```
Please assess the credibility of these conflicting research findings:

---
## Tesla Stock Analysis - Conflicting Views

**Bullish Research (Source: Tesla fan blog "TeslaDaily.com"):**
- "Tesla will dominate the robotaxi market by 2025"
- "Full Self-Driving is already safer than human drivers"
- "Tesla's margins are industry-leading and sustainable"
- "Competition from legacy automakers is irrelevant"

**Bearish Research (Source: Short-seller report from "Hindenburg Research"):**
- "Tesla's FSD claims are exaggerated and potentially dangerous"
- "Margins are declining due to price cuts and competition"
- "Chinese EV makers are taking significant market share"
- "Robotaxi timeline has been repeatedly delayed"

**Neutral Research (Source: Goldman Sachs equity research):**
- "Tesla maintains technology leadership but faces margin pressure"
- "FSD progress is notable but regulatory approval timeline uncertain"
- "Competition is intensifying but Tesla's brand remains strong"

**Original Question:** "What is the outlook for Tesla stock over the next 12 months?"
---

Please:
1. Assess the credibility of each source
2. Identify which claims can be verified vs which are speculative
3. Note any potential biases in each source
4. Determine if this research provides a balanced view to answer the original question
```
---

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/website_quality.txt'}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/research_lessons.txt'}
```

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/source_notes.txt'}
```

### 🔧 Tool Call: `web_search`
```json
{'query': 'Tesla FSD safety vs human drivers crash rate', 'max_results': 3, 'topic': 'general'}
```

...

### 🔧 Tool Call: `read_file`
```json
{'file_path': '/memories/research_lessons.txt'}
```

### 📤 Tool Response
```
     1	## Research & Analysis Lessons
     2	- Use clear, minimal matplotlib styles and always report saved figure paths for users.
     3	- For “largest company by market cap” checks, prefer same-day rankings from major financial data providers (Bloomberg, Refinitiv, Yahoo Finance) and note that leadership can change intraday.
     4	- When assessing company performance claims, verify timeframes (e.g., "in 2024", "most recent quarter") against actual reported periods to avoid mismatch.
```
---

### 🔧 Tool Call: `edit_file`
```json
{'file_path': '/memories/research_lessons.txt', 'old_string': '- When assessing company performance claims, verify timeframes (e.g., "in 2024", "most recent quarter") against actual reported periods to avoid mismatch.', 'new_string': '- When assessing company performance claims, verify timeframes (e.g., "in 2024", "most recent quarter") against actual reported periods to avoid mismatch.\n- For controversial tech-safety claims (e.g., FSD vs human safety), distinguish between company-reported stat
...
```

### 📤 Tool Response
```
Successfully replaced 1 instance(s) of the string in '/research_lessons.txt'
```
---

## ✅ Response
### Claim Verification

**Bullish (TeslaDaily fan blog)**  
- “Tesla will dominate the robotaxi market by 2025”: **UNVERIFIED / HIGHLY SPECULATIVE**  
  - Evidence: No approved, large-scale commercial Tesla robotaxi service exists yet; timelines have been repeatedly pushed out. Regulatory and technical uncertainty is large.  
- “FSD is already safer than human drivers”: **PARTIALLY VERIFIED / MISLEADING**  
  - Evidence: Tesla publishes crash-rate stats suggesting fewer crashes per mile with Autopilot/FSD engaged, but these are company-selected metrics and not independently peer-reviewed. Experts (e.g., Phil Koopman) note apples‑to‑oranges comparisons and lack of transparent baselines.  
- “Tesla's margins are industry-leading and sustainable”: **PARTIALLY VERIFIED**  
  - Evidence: Tesla historically had very high auto gross margins, but data show margins fell from ~25–26% (2022) to the mid‑teens by 2023–24 due to price cuts and competition (e.g., Yahoo Finance/Forbes coverage). “Sustainable” is speculative.  
- “Competition from legacy automakers is irrelevant”: **CONTRADICTED**  
  - Evidence: Legacy and Chinese automakers have grown EV share; price cuts and slower Tesla growth suggest competition is impacting pricing and margins.

**Bearish (Hindenburg short-seller report)**  
- “Tesla's FSD claims are exaggerated and potentially dangerous”: **PARTIALLY VERIFIED**  
  - Evidence: Regulators (NHTSA) have investigated Autopilot/FSD; there have been high-profile crashes and recalls. Some marketing language (e.g., “Full Self‑Driving”) is widely criticized as overpromising. “Potentially dangerous” reflects real safety concerns but also carries rhetorical bias.  
- “Margins are declining due to price cuts and competition”: **VERIFIED**  
  - Evidence: Reported financials show declining auto gross margins 2022–2024; commentary widely attributes this to price cuts amid rising competition.  
- “Chinese EV makers are taking significant market share”: **VERIFIED**  
  - Evidence: BYD and others have rapidly increased EV sales and are directly competing with Tesla in China and globally.  
- “Robotaxi timeline has been repeatedly delayed”: **VERIFIED**  
  - Evidence: Public statements by Tesla/Elon Musk have repeatedly moved expectations for full autonomy/robotaxis over many years.

**Neutral (Goldman Sachs equity research)**  
- “Tesla maintains technology leadership but faces margin pressure”: **PARTIALLY VERIFIED**  
  - Evidence: Widely viewed as a tech leader in EVs and software, while margins have clearly compressed. Leadership is somewhat subjective but broadly accepted.  
- “FSD progress notable but regulatory approval timeline uncertain”: **VERIFIED (as a characterization)**  
  - Evidence: Software iterations are ongoing; no clear regulatory green light for unsupervised robotaxis in most markets.  
- “Competition is intensifying but Tesla's brand remains strong”: **VERIFIED (directionally)**  
  - Evidence: EV competition has intensified; Tesla brand strength is supported by sales scale and awareness, though sentiment is mixed.

---

### Source Assessment

- **TeslaDaily (fan blog)**: **2/5**  
  - Strong bullish bias; non-institutional; likely to overemphasize positives and downplay risks. Limited transparency on methodology; mostly speculative forward-looking claims.

- **Hindenburg Research (short-seller)**: **2/5**  
  - Investigative work can surface real issues, but they are financially incentivized for negative outcomes. Tends to emphasize worst-case interpretations and may underweight positives.

- **Goldman Sachs equity research**: **4/5**  
  - Regulated institution with compliance processes and access to company/industry data. Still may have investment‑banking/conflict considerations, but generally more balanced and data‑driven.

---

### Answer Quality vs Original Question (12‑month Tesla stock outlook)

- The **bullish** and **bearish** pieces are heavy on **long-term, speculative narratives** (robotaxis dominance or failure, extreme safety claims) and relatively light on concrete 12‑month drivers (earnings, valuations, macro, rates, regulatory milestones).
- The **Goldman Sachs** summary is closest to a balanced 12‑month framing (tech lead + margin pressure + competition), but you’d still need: valuation context, base‑case earnings, and explicit upside/downside scenarios.

**Does this research provide a balanced view?**  
- **Partially.** You do see a spectrum: optimistic, pessimistic, and neutral.  
- However:
  - Extremes (fan blog, short-seller) are both materially biased.
  - Key 12‑month factors (valuation, interest rates, demand elasticity, actual earnings guidance) are missing.
  - Robotaxi/FSD dominance is **too speculative** to anchor a 12‑month stock outlook.

---

### Recommendations

- **Weight sources**: Put primary weight on institutional/neutral research (e.g., Goldman, other major brokers) and Tesla’s own filings; treat fan blog and short-seller narratives as sentiment extremes.  
- **Separate**:
  - **Verifiable near-term facts**: current margins, pricing, deliveries, capex, regulatory actions.
  - **Speculative long-term stories**: robotaxis dominance, total obsolescence due to competition.  
- **For a 12‑month view**, focus on: margin trajectory, volume growth, pricing power, competitive response, FSD monetization pace (actual revenue, not promises), and valuation vs peers.

### Final Verdict

- **Trustworthy and defensible?**  
  - As a **complete basis for a 12‑month outlook**: **With caveats / Needs supplementation**.  
  - Use the bullish/bearish pieces to understand sentiment and risk narratives, but anchor decisions in institutional research and hard financial/regulatory data.

---
# Notes

## Expected Outputs

For each example, the credibility agent should provide:

### Claim Verification
- VERIFIED / PARTIALLY VERIFIED / UNVERIFIED / CONTRADICTED status for each claim
- Evidence supporting the verification status

### Source Assessment
- Overall source quality rating (1-5)
- Concerns about any sources

### Answer Quality
- Whether the research answers the original question
- Missing elements

### Recommendations
- Suggested corrections or additions
- Areas needing more research

### Final Verdict
- Trustworthy and defensible? (Yes/With caveats/Needs work)

## Credibility Criteria Used

**High Credibility Sources:**
- Peer-reviewed research
- Official government/institutional data
- Established news organizations
- Primary sources and original documents

**Lower Credibility Sources:**
- Blogs and opinion pieces
- Social media
- Sites with heavy advertising
- Sources with clear conflicts of interest

## Integration with Main Agent

In production, the main agent would:
1. Receive research from `web-research-agent`
2. Send research to `credibility-agent` for verification
3. Use credibility feedback to refine or supplement research
4. Only include verified information in final reports